In [ ]:
# Import Splinter, BeautifulSoup and dependencies
import requests
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
from splinter import Browser
from selenium import webdriver
import time
import pandas as pd
from pprint import pprint
import numpy as np

In [ ]:
# Launch browser
#browser = Browser('chrome')
#time.sleep(2)
#url = "https://nuforc.org/subndx/?id=highlights"
#browser.visit(url)

In [ ]:
# Create Beautiful Soup
#html = browser.html
#soup = bs(html, 'html.parser')
#table =  soup.find('div', id="outer-wrap").find('div', id="table_1_wrapper").find('table', id="table_1")

In [ ]:
#sightings = []

In [ ]:
#odd_rows = table.find_all('tr', class_='odd')
#for odd in odd_rows:
#    data = odd.find_all('td')
#    odd = [odd.text for odd in data]
#    sightings.append(odd)

In [ ]:
#even_rows = table.find_all('tr', class_='even')
#for even in even_rows:
#    data = even.find_all('td')
#    even = [even.text for even in data]
#    sightings.append(even)

In [ ]:
# Launch browser
browser = Browser('chrome') 
url = "https://nuforc.org/subndx/?id=all"
browser.visit(url)

# Initialize an empty list to store all sightings
sightings = []


In [ ]:
# Iterate through each page
while True:
    # Create Beautiful Soup object
    html = browser.html
    soup = bs(html, 'html.parser')
    
    time.sleep(2) 
    # Find the sightings data table on the current page
    table = soup.find('div', id='outer-wrap').find('table', id='table_1')

    time.sleep(2)
    # Extract odd rows class
    odd_rows = table.find_all('tr', class_='odd')
    for odd in odd_rows:
        data = odd.find_all('td')
        odd_data = [d.text for d in data]
        sightings.append(odd_data)
        print(odd_data)
    
    time.sleep(2)
    # Extract even rows class
    even_rows = table.find_all('tr', class_='even')
    for even in even_rows:
        data = even.find_all('td')
        even_data = [d.text for d in data]
        sightings.append(even_data)
        print(even_data)

    time.sleep(2)
    # Click the next button using JavaScript to bypass interception
    try:
        next_button = browser.find_by_id('table_1_next')
        browser.execute_script("arguments[0].click();", next_button[0]._element)
    except Exception as e:
        print(f"Error clicking next button: {e}")
        break  

In [ ]:
# Check if there is a next button
    next_button = browser.find_by_id('table_1_next')
    if not next_button.is_empty():
        next_button.click()
        time.sleep(5)  # Allow time for the next page to load
    else:
        break  # Exit the loop if there's no next button

In [ ]:
# Close the browser
browser.quit()


In [ ]:
# Print sightings data
for sighting in sightings:
    print(sighting)

In [ ]:
sightings_df = pd.DataFrame(sightings,
                           columns = ["LINK",
                                     "OCCURRED_DATE",
                                     "CITY",
                                     "STATE",
                                     "COUNTRY",
                                     "SHAPE",
                                     "SUMMARY",
                                     "MEDIA",
                                     "EXPLANATION"
                                     ])
sightings_df

In [ ]:
sightings_df = sightings_df.drop(columns = ["LINK", "MEDIA", "EXPLANATION"])
sightings_df

In [ ]:
sightings_df["OCCURRED_DATE"] = pd.to_datetime(sightings_df["OCCURRED_DATE"]).dt.date
sightings_df

In [ ]:
usa_sightings_df = sightings_df.loc[sightings_df['COUNTRY'] == 'USA']
usa_sightings_df['CITY'] = usa_sightings_df['CITY'].replace('', np.nan)
usa_sightings_df = usa_sightings_df.dropna()
usa_sightings_df['CITYSTATE'] = usa_sightings_df['CITY'].str.cat(usa_sightings_df['STATE'], sep=', ')
usa_sightings_df